In [13]:
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore")
import pmdarima
from pmdarima import auto_arima
from math import sqrt
from sklearn.metrics import mean_squared_error
from composition_stats import closure

In [14]:
import os 
  
# current directory 
current_dir = os.getcwd() 
relative_path=os.path.abspath(os.path.join(current_dir, os.pardir))

In [39]:

relative_path

'c:\\Users\\User\\Projects\\FYP_8'

In [15]:
train_data= pd.DataFrame(pd.read_csv(
    relative_path+"\\age_structure_data\\df_train.csv"))
train_data['Year'] = pd.to_datetime(train_data['Year'])
train_data = train_data.set_index("Year")

test_data= pd.DataFrame(pd.read_csv(
    relative_path+"\\age_structure_data\\df_test.csv"))
test_data['Year'] = pd.to_datetime(test_data['Year'])
test_data = test_data.set_index("Year")

In [16]:
e_ori=train_data['elder']
y_ori=train_data['youth']
k_ori=train_data['kid']

In [17]:
def calculate_rmse_for_cv(train_data, test_data, a, b, c):
    arima_model = ARIMA(train_data, order=(a, b, c)).fit()
    prediction = arima_model.predict(
        start=len(train_data), end=len(train_data)+len(test_data)-1)
    rmse = sqrt(mean_squared_error(test_data, prediction))
    return rmse, test_data, prediction

In [18]:
auto_arima(e_ori, start_p=0, d=2 ,start_q=0,
           max_p=10, max_q=10, m=1, trace=True, stepwise=True, seasonal=False)

Performing stepwise search to minimize aic
 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=-617.256, Time=0.18 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=-617.953, Time=0.24 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=-618.910, Time=0.18 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=-618.390, Time=0.02 sec
 ARIMA(1,2,1)(0,0,0)[0] intercept   : AIC=-616.949, Time=0.20 sec
 ARIMA(0,2,2)(0,0,0)[0] intercept   : AIC=-616.620, Time=0.20 sec
 ARIMA(1,2,2)(0,0,0)[0] intercept   : AIC=-614.791, Time=0.22 sec
 ARIMA(0,2,1)(0,0,0)[0]             : AIC=-620.310, Time=0.09 sec
 ARIMA(1,2,1)(0,0,0)[0]             : AIC=-618.383, Time=0.27 sec
 ARIMA(0,2,2)(0,0,0)[0]             : AIC=-618.073, Time=0.13 sec
 ARIMA(1,2,0)(0,0,0)[0]             : AIC=-619.278, Time=0.11 sec
 ARIMA(1,2,2)(0,0,0)[0]             : AIC=-616.108, Time=0.18 sec

Best model:  ARIMA(0,2,1)(0,0,0)[0]          
Total fit time: 2.031 seconds


ARIMA(order=(0, 2, 1), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [19]:
train_e_1, test_e_1 = train_test_split(e_ori, test_size=0.2, shuffle=False)
train_e_2, test_e_2 = train_test_split(train_e_1, test_size=0.2, shuffle=False)
train_e_3, test_e_3 = train_test_split(train_e_2, test_size=0.2, shuffle=False)
rmse_e_1, test_data_e_1, prediction_test_e_1=calculate_rmse_for_cv(train_e_1, test_e_1,0,2,1)
rmse_e_2, test_data_e_2, prediction_test_e_2=calculate_rmse_for_cv(train_e_2, test_e_2,0,2,1)
rmse_e_3, test_data_e_3, prediction_test_e_3=calculate_rmse_for_cv(train_e_3, test_e_3,0,2,1)
average_rmse_e=(rmse_e_1+rmse_e_2+rmse_e_3)/3
average_rmse_e

0.0018668344804598483

In [20]:
arima_model_e = ARIMA(e_ori, order=(0, 2, 1)).fit()
prediction_e = arima_model_e.predict(
    start=len(e_ori), end=len(e_ori)+len(test_data)-1)
prediction_e = prediction_e.to_frame()
prediction_e.rename(columns = {'predicted_mean':0}, inplace = True)
final_e = pd.concat([e_ori, prediction_e])

In [21]:
auto_arima(y_ori, start_p=0, d=1 ,start_q=0,
           max_p=10, max_d=2, max_q=10, m=1, trace=True, stepwise=True, seasonal=False)

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-472.451, Time=0.15 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-503.713, Time=0.06 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-496.202, Time=0.21 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-372.031, Time=0.03 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=-503.877, Time=0.14 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=-502.041, Time=0.15 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-501.104, Time=0.09 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-503.614, Time=0.10 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=-499.077, Time=0.22 sec
 ARIMA(2,1,0)(0,0,0)[0]             : AIC=-496.220, Time=0.05 sec

Best model:  ARIMA(2,1,0)(0,0,0)[0] intercept
Total fit time: 1.200 seconds


ARIMA(order=(2, 1, 0), scoring_args={}, suppress_warnings=True)

In [22]:
train_y_1, test_y_1 = train_test_split(y_ori, test_size=0.2, shuffle=False)
train_y_2, test_y_2 = train_test_split(train_y_1, test_size=0.2, shuffle=False)
train_y_3, test_y_3 = train_test_split(train_y_2, test_size=0.2, shuffle=False)
rmse_y_1, test_data_y_1, prediction_test_y_1=calculate_rmse_for_cv(train_y_1, test_y_1, 2, 1, 0)
rmse_y_2, test_data_y_2, prediction_test_y_2=calculate_rmse_for_cv(train_y_2, test_y_2, 2, 1, 0)
rmse_y_3, test_data_y_3, prediction_test_y_3=calculate_rmse_for_cv(train_y_3, test_y_3, 2, 1, 0)
 
average_rmse_y=(rmse_y_1+rmse_y_2+rmse_y_3)/3
average_rmse_y

0.0030185150383896914

In [23]:
arima_model_y = ARIMA(y_ori, order=(2, 1, 0)).fit()
prediction_y = arima_model_y.predict(
    start=len(y_ori), end=len(y_ori)+len(test_data)-1)
prediction_y = prediction_y.to_frame()
prediction_y.rename(columns = {'predicted_mean':0}, inplace = True)
final_y = pd.concat([y_ori, prediction_y])

In [24]:
auto_arima(k_ori, start_p=0, d=2 ,start_q=0,
           max_p=10, max_d=2, max_q=10, m=1, trace=True, stepwise=True, seasonal=False)

Performing stepwise search to minimize aic
 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=-497.492, Time=0.08 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=-495.780, Time=0.07 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=-495.145, Time=0.07 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=-498.903, Time=0.02 sec
 ARIMA(1,2,1)(0,0,0)[0] intercept   : AIC=-493.117, Time=0.09 sec

Best model:  ARIMA(0,2,0)(0,0,0)[0]          
Total fit time: 0.330 seconds


ARIMA(order=(0, 2, 0), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [25]:
train_k_1, test_k_1 = train_test_split(k_ori, test_size=0.2, shuffle=False)
train_k_2, test_k_2 = train_test_split(train_k_1, test_size=0.2, shuffle=False)
train_k_3, test_k_3 = train_test_split(train_k_2, test_size=0.2, shuffle=False)
rmse_k_1, test_data_k_1, prediction_test_k_1=calculate_rmse_for_cv(train_k_1, test_k_1, 0, 2, 0)
rmse_k_2, test_data_k_2, prediction_test_k_2=calculate_rmse_for_cv(train_k_2, test_k_2, 0, 2, 0)
rmse_k_3, test_data_k_3, prediction_test_k_3=calculate_rmse_for_cv(train_k_3, test_k_3, 0, 2, 0)
 
average_rmse_k=(rmse_k_1+rmse_k_2+rmse_k_3)/3
average_rmse_k

0.00314313693197048

In [26]:
arima_model_k = ARIMA(k_ori, order=(0, 2, 0)).fit()
prediction_k = arima_model_k.predict(
    start=len(k_ori), end=len(k_ori)+len(test_data)-1)
prediction_k = prediction_k.to_frame()
prediction_k.rename(columns = {'predicted_mean':0}, inplace = True)
final_k = pd.concat([k_ori, prediction_k])

In [27]:
final_e.rename(columns = {0:'e'}, inplace = True)
final_y.rename(columns = {0:'y'}, inplace = True)
final_k.rename(columns = {0:'k'}, inplace = True)

In [28]:
full = pd.concat([final_e, final_y,final_k],axis=1)

In [29]:
data_proportion = closure(full)

In [30]:
def set_col(df):
    df = pd.DataFrame(df, columns=[
    'elder', 'youth', 'kid'])
    df['Year'] = df.index + 1964
    df['Year'] = pd.to_datetime(df['Year'], format='%Y')
    df = df.set_index("Year")
    return df

In [31]:
proportion = set_col(data_proportion)


In [32]:
df_actual=proportion[(proportion.index< '2010-01-01')]
df_forecast=proportion[(proportion.index> '2009-01-01')]

In [33]:
proportion_actual_1=pd.concat([test_data_e_1.to_frame(), test_data_y_1.to_frame(),  test_data_k_1.to_frame()],axis=1)
proportion_actual_2=pd.concat([test_data_e_2.to_frame(), test_data_y_2.to_frame(), test_data_k_2.to_frame()],axis=1)
proportion_actual_3=pd.concat([test_data_e_3.to_frame(), test_data_y_3.to_frame(), test_data_k_3.to_frame()],axis=1)
proportion_forecast_1=closure(pd.concat([prediction_test_e_1.to_frame(), prediction_test_y_1.to_frame(), prediction_test_k_1.to_frame()],axis=1))
proportion_forecast_2=closure(pd.concat([prediction_test_e_2.to_frame(), prediction_test_y_2.to_frame(),prediction_test_k_2.to_frame()],axis=1))
proportion_forecast_3=closure(pd.concat([prediction_test_e_3.to_frame(), prediction_test_y_3.to_frame(),prediction_test_k_3.to_frame()],axis=1))

In [34]:
rmse1 = sqrt(mean_squared_error(proportion_actual_1, proportion_forecast_1))
rmse2 = sqrt(mean_squared_error(proportion_actual_2, proportion_forecast_2))
rmse3 = sqrt(mean_squared_error(proportion_actual_3, proportion_forecast_3))

rmse_cv=(rmse1+rmse2+rmse3)/3
rmse_cv

0.0029791632517698727

In [35]:
rmse_final = sqrt(mean_squared_error(df_forecast, test_data))
rmse_final

0.009235412959796033

In [36]:
df_forecast.to_csv(relative_path+"\\age_structure_data\\auto_base_forecast.csv")

In [37]:
full['added_together']=full['e']+full['y']+full['k']

In [38]:
full.head()

,e,y,k,added_together
1964-01-01,0.032504,0.504384,0.463112,1.0
1965-01-01,0.031992,0.506028,0.461980,1.0
1966-01-01,0.032161,0.507334,0.460504,1.0
1967-01-01,0.032281,0.511010,0.456709,1.0
1968-01-01,0.032420,0.515973,0.451607,1.0
